# Aerodynamic Wing Optimization Design Problem

## Brief overview of the problem

The aerodynamic wing optimization design problem involves optimizing the shape of a rectangular wing to minimize drag and achieve a specific lift coefficient. The problem uses an open-source software package called [OpenMDAO](https://openmdao.org/) to create a multi-disciplinary optimization framework that combines aerodynamics, structural mechanics, and geometric design.

The optimization problem takes as input several design parameters such as the chord and span of the wing, and the angle of attack, and uses an aerodynamic solver to compute the lift and drag coefficients of the wing at a specific flight condition. The solver is based on a Vortex Lattice Method (VLM) that approximates the aerodynamics of the wing using a discrete set of vortices.

The optimization algorithm then adjusts the design parameters to find the set of parameters that minimizes the drag of the wing while achieving a specific lift coefficient. The algorithm uses the [SNOPT optimization solver](http://www.sbsi-sol-optimize.com/asp/sol_product_snopt.htm) to find the optimal solution.

The problem can be modified to explore different design objectives, such as optimizing for both lift and stability, or to include additional disciplines such as structural optimization or uncertainty quantification. The example provides a starting point for engineers and scientists to use open-source software and computational methods to optimize the design of a wing and improve its performance.

## Tools used in this example

This code example demonstrates the use of several tools commonly used in aerospace engineering design and analysis: OpenMDAO, OpenVSP, and VSPAero.

[OpenMDAO](https://openmdao.org/) is an open-source software framework for multidisciplinary design optimization (MDO) of complex systems. It provides a framework for building and solving optimization problems with a modular approach. It allows for the integration of different disciplines and optimization algorithms, making it a valuable tool for aerospace engineering design problems that often require the consideration of multiple disciplines.

[OpenVSP](https://openvsp.org/) is an open-source parametric aircraft geometry tool used to create 3D models of aircraft. It allows for the creation of complex geometry models and provides tools for analysis and optimization. OpenVSP was used in this example to create the geometry of a rectangular wing.

[VSPAero](https://github.com/OpenMDAO/VSPAERO) is an aerodynamic analysis tool that works with OpenVSP to provide aerodynamic analysis and optimization capabilities. VSPAero uses the Vortex Lattice Method (VLM) to compute the aerodynamic forces and moments on the aircraft. In this example, VSPAero was used to perform an aerodynamic analysis of the rectangular wing created in OpenVSP. The analysis was performed for a specific flight condition defined by the angle of attack and wing area.

The use of these tools in this example demonstrates a typical workflow in aerospace engineering design and analysis. OpenVSP is used to create the aircraft geometry, which is then analyzed using VSPAero. OpenMDAO is used to integrate the aerodynamic analysis results into an optimization problem to find the optimal design of the aircraft under certain constraints. This workflow enables engineers to explore the design space and find optimal solutions that meet the given design requirements.

## Run script and explanation

This code block defines an OpenMDAO component called FlightCase, which sets up and solves an aerodynamic analysis problem using the vortex lattice method (VLM) for a given aircraft design, specified using OpenVSP.

The component first initializes the VLM assembler pyVSPAero with the provided VSP model file. The VLM assembler sets up the simulation and provides the necessary functions to compute aerodynamic coefficients CL and CD. These functions are added as outputs to the component.

In the setup method, the component sets up its inputs, which include the angle of attack aoa, reference area S_ref, and the initial aircraft geometry x_aero. It also sets up the outputs for the aerodynamic coefficients. Additionally, the component sets up the VLMAssembler object by creating a steady problem instance with the specified name and options.

The compute method of the component sets the flight variables and geometry using the inputs, solves the VLM problem, and evaluates the output functions to get the values of the aerodynamic coefficients. These coefficients are assigned to the component's outputs.

The compute_partials method computes the partial derivatives of the output functions with respect to the input variables. These partial derivatives are used in the gradient-based optimization algorithm to find the optimal design.

In [ ]:
import os
import openmdao.api as om
from pygeo.mphys import OM_DVGEOCOMP
import openvsp

from vspaero.pyvspaero import pyVSPAero
from vspaero import functions


class FlightCase(om.ExplicitComponent):
    def initialize(self):
        self.options.declare("vsp_file", recordable=False)

    def setup(self):
        # VSP model
        vsp_file = self.options["vsp_file"]

        VLMAssembler = pyVSPAero(vsp_file)
        # Add output functions
        VLMAssembler.add_function("CL", functions.CLi)
        VLMAssembler.add_function("CD", functions.CDi)
        self.assembler = VLMAssembler

        # Create a steady problem instance
        self.problem = VLMAssembler.create_steady_problem(self.name, options={"symmetry": "Y"})

        mesh_xyz = self.problem.get_geometry()

        self.add_input("aoa", val=5.0)
        self.add_input("S_ref", val=10.0)
        self.add_input("x_aero", val=mesh_xyz, shape=mesh_xyz.shape)
        for out_name in self.problem.func_list:
            self.add_output(out_name, val=1.0)

        self.declare_partials("*", "*")

    def get_mesh(self):
        return self.problem.get_geometry()

    def compute(self, inputs, outputs):
        self.problem.set_flight_vars(**inputs)
        self.problem.set_geometry(inputs["x_aero"])

        self.problem.solve()

        funcs = {}
        self.problem.eval_functions(funcs)

        for out_name in outputs:
            output_key = f"{self.name}_{out_name}"
            outputs[out_name] = funcs[output_key]

        self.problem.write_solution(output_dir="./output")

    def compute_partials(self, inputs, J):
        """ Jacobian of partial derivatives."""

        self.problem.set_flight_vars(**inputs)
        self.problem.set_geometry(inputs["x_aero"])

        funcs_sens = {}
        self.problem.eval_functions_sens(funcs_sens)

        for out_name in self.problem.func_list:
            for in_name in inputs:
                output_key = f"{self.name}_{out_name}"
                if in_name == "x_aero":
                    J[out_name, in_name] = funcs_sens[output_key]["xyz"]
                else:
                    J[out_name, in_name] = funcs_sens[output_key][in_name]

The Planform class is an ExplicitComponent that calculates the reference wing area (S_ref) of a VSP model. S_ref is the reference area used to calculate lift and drag forces. The class takes WingGeom parameters (root chord and tip chord for each section of the wing) as inputs and returns the reference area as output. The reference wing area is calculated using the openvsp package, which is used to read and write VSP files. The compute function takes the inputs dictionary containing the WingGeom parameters as inputs, updates the VSP model with the new values, calculates the reference area using GetParmVal function from openvsp, and sets the reference area as the output of the component.

In [ ]:
class Planform(om.ExplicitComponent):

    def initialize(self):
        self.options.declare("vsp_file", recordable=False)

    def setup(self):
        # VSP model
        vsp_file = self.options["vsp_file"]
        self.add_input("WingGeom:XSec_1:Root_Chord", val=1)
        self.add_input("WingGeom:XSec_2:Root_Chord", val=1)
        self.add_input("WingGeom:XSec_3:Root_Chord", val=1)
        self.add_input("WingGeom:XSec_4:Root_Chord", val=1)
        self.add_input("WingGeom:XSec_4:Tip_Chord", val=1)
        self.add_output("S_ref", val=1.0)
        self.declare_partials("S_ref", "*", method="fd")
        self.vsp_model = openvsp.VSPVehicle(vsp_file)

    def compute(self, inputs, outputs):
        for input_key in inputs:
            val = inputs[input_key][0]
            geom_name, group_name, parm_name = input_key.split(":")
            container_id = self.vsp_model.FindContainer(geom_name, 0)
            self.vsp_model.SetParmValUpdate(container_id, parm_name, group_name, val)
        self.vsp_model.Update()
        parm_id = self.vsp_model.FindParm(container_id, "TotalArea", geom_name)
        outputs["S_ref"] = self.vsp_model.GetParmVal(parm_id)

Top is an om.Group object that is used to collect and manage several components of the optimization problem.

The setup method of Top is used to set up the optimization problem. It adds several subsystems to the problem:

An initial_mesh subsystem that initializes the mesh.
A geometry subsystem that adds geometry to the problem.
A planform subsystem that calculates the planform.
A cruise subsystem that calculates the cruise performance.
The initial_mesh and geometry subsystems set up the geometry of the problem using OM_DVGEOCOMP and a vsp_file. geometry adds a pointset to the geometry called "aero" that will be used in the cruise analysis.

The planform subsystem calculates the planform of the problem. It sets up design variables for the root and tip chord lengths for each of the four wing sections, and computes the wing reference area.

The cruise subsystem calculates the performance of the wing in cruise mode using FlightCase which sets up the VLM model and computes the CL and CD.

The optimization problem is then set up by creating an instance of om.Problem(), adding the Top() group to the problem, setting up the driver as om.pyOptSparseDriver, and adding the design variables, constraints, and objective.

The problem is then run by calling prob.run_driver() and the results are printed.

In [ ]:
class Top(om.Group):
    def setup(self):
        
        # Setup VLM assembler
        vsp_file = "rect_wing.vsp3"
        cruise_component = FlightCase(vsp_file=vsp_file)

        self.add_subsystem("initial_mesh", om.IndepVarComp())

        # add the geometry component, we dont need a builder because we do it here.
        self.add_subsystem("geometry", OM_DVGEOCOMP(file=vsp_file, type="vsp"), promotes_inputs=["WingGeom*"])
        # add pointset
        self.geometry.nom_add_discipline_coords("aero")

        self.add_subsystem("planform", Planform(vsp_file=vsp_file), promotes_inputs=['*'])
        self.connect("planform.S_ref", "cruise.S_ref")

        self.add_subsystem("cruise", cruise_component)
        self.connect("geometry.x_aero0", "cruise.x_aero")

    def configure(self):
        self.initial_mesh.add_output("x_aero0", self.cruise.get_mesh())
        self.connect("initial_mesh.x_aero0", "geometry.x_aero_in")

        # create geometric DV setup
        self.geometry.nom_addVSPVariable("WingGeom", "XSec_1", "Root_Chord", scaledStep=False)
        self.geometry.nom_addVSPVariable("WingGeom", "XSec_2", "Root_Chord", scaledStep=False)
        self.geometry.nom_addVSPVariable("WingGeom", "XSec_3", "Root_Chord", scaledStep=False)
        self.geometry.nom_addVSPVariable("WingGeom", "XSec_4", "Root_Chord", scaledStep=False)
        self.geometry.nom_addVSPVariable("WingGeom", "XSec_4", "Tip_Chord", scaledStep=False)

prob = om.Problem()
prob.model = Top()

# Set optimizer as model driver
prob.driver = om.pyOptSparseDriver(debug_print=['objs', 'nl_cons', "desvars"])
prob.driver.options['optimizer'] = "SNOPT"
prob.driver.opt_settings['Major iterations limit'] = 200
prob.driver.opt_settings["Nonderivative linesearch"] = None
prob.driver.opt_settings["Major optimality tolerance"] = 1e-4
prob.driver.opt_settings["Major feasibility tolerance"] = 1e-5
prob.driver.opt_settings["Hessian updates"] = 10


# Setup problem and add design variables, constraint, and objective
prob.model.add_design_var("WingGeom:XSec_1:Root_Chord", lower=1e-3, upper=5.0)
prob.model.add_design_var("WingGeom:XSec_2:Root_Chord", lower=1e-3, upper=5.0)
prob.model.add_design_var("WingGeom:XSec_3:Root_Chord", lower=1e-3, upper=5.0)
prob.model.add_design_var("WingGeom:XSec_4:Root_Chord", lower=1e-3, upper=5.0)
prob.model.add_design_var("WingGeom:XSec_4:Tip_Chord", lower=1e-3, upper=5.0)
prob.model.add_design_var("cruise.aoa", lower=-10.0, upper=10.0)
prob.model.add_constraint("cruise.CL", equals=0.5)
prob.model.add_constraint("planform.S_ref", equals=10.0, scaler=0.1)
prob.model.add_objective("cruise.CD", scaler=1e4)

prob.setup()

# Create a n^2 diagram for user to view model connections
om.n2(prob, show_browser=False)
prob.run_model()
print("CL", prob["cruise.CL"][0])
print("CD", prob["cruise.CD"][0])

prob.run_driver()


Initializing DVGeometryVSP
Loading the vsp model took: 0.03481459617614746
Building a quad mesh for fast projections.
Initialized DVGeometry VSP in 0.05066394805908203 seconds.


DVGeometryVSP note:
Adding pointset x_aero0 took 0.003933429718017578 seconds.
Maximum distance between the added points and the VSP geometry is 3.731756137924373e-12
CL 0.429910394713496
CD 0.005992091473352069

+------------------------------------------------------------------------------+
| pyOptSparse Error: Failed to properly open SNOPT_print.out, ierror = 5002    |
+------------------------------------------------------------------------------+



Error: Failed to properly open SNOPT_print.out, ierror = 5002